****1**** ***Importación de modulos requeridos***

In [1]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime
import requests
from zipfile import ZipFile

****2**** ***Datos utilizados***

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
with ZipFile('datasource.zip', 'r') as zip:
        zip.extractall('dealership_data')


--2022-01-12 22:47:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-01-12 22:47:33 (391 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [3]:
tmpfile = "dealership_temp.tmp" # store all extracted data
logfile = "dealership_logfile.txt" # all event logs will be stored
targetfile = "dealership_transformed_data.csv" # transformed data is stored

****3****  ***Extracción de datos***

****3.1****  ***Extraer CSV***

In [4]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

****3.2****  ***Extraer JSON***

In [5]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

****3.3****  ***Extraer XML***

In [6]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price',
'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model,
"year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel},
ignore_index=True)
        return dataframe

****3.4****  ***Extraccion***

In [7]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
#for csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
            extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
#for json files
    for jsonfile in glob.glob("dealership_data/*.json"):
            extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
#for xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
            extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
    return extracted_data

****4****  ***Transformar***

In [8]:
def transform(data):
    data['price'] = round(data.price, 2)
    return data

****5****  ***Carga y Registro***

In [9]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [10]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f: f.write(timestamp + ',' + message +'n')

****6****  ***Ejecución***

In [11]:
log("ETL Job Started")

In [12]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

In [13]:
log("Transform phase Ended")
transformed_data = transform(extracted_data)

In [14]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [15]:
log("ETL Job Ended")